# INFO 4271 - Group Project

Issued: June 11, 2024

Due: July 22, 2024

Please submit a link to your code base (ideally with a branch that does not change anymore after the submission deadline) and your 4-page report via email to carsten.eickhoff@uni-tuebingen.de by the due date. One submission per team.

---

# 1. Web Crawling & Indexing
Crawl the web to discover **English content related to Tübingen**. The crawled content should be stored locally. If interrupted, your crawler should be able to re-start and pick up the crawling process at any time.

In [ ]:
#### Imports ####
import requests
import copy
from boilerpy3 import extractors
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag, download
import re
from trafilatura import fetch_url, extract
import contractions
from nltk.stem import WordNetLemmatizer
from duplicateCheck import check_simhash,computeHash
import math
import numpy as np
from sklearn.cluster import KMeans
import json
import os
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.robotparser import RobotFileParser as rp
from bs4 import BeautifulSoup as bs
import html5lib
import requests
import stopit
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from langdetect import detect
import random
from flask import Flask, render_template, request, redirect, url_for
import logging
from collections import Counter
from spellchecker import SpellChecker
# Load the German and english dictionary
spellGerman = SpellChecker(language='de')
spellEnglish = SpellChecker(language='en')


In [4]:
##returns corresponding wordnet POS tag for a nltk pos tag
# returns none if no lemmatization necessary
def getWordnetPos(nltkPos):
   if nltkPos.startswith('J'):
        return wordnet.ADJ
   elif nltkPos.startswith('V'):
        return wordnet.VERB
   elif nltkPos.startswith('N'):
        return wordnet.NOUN
   elif nltkPos.startswith('R'):
        return wordnet.ADV
   else:
        return None
    

#### Preprocessing of terms ####
##preprocesses a term t 
#returns preprocessed t or False in case term is uninformative
def preprocess(t, tag):
 #get nltk stopwords english and german! (often also german stopwords still contained)
 stop_words = set(stopwords.words('english')).union(set(stopwords.words('german')))

 #initialize lemmatizer
 lemmatizer = WordNetLemmatizer()

 #convert to lowercase
 t = t.lower()

 #is url (starting with http(s) or www.) or empty
 if re.compile(r'https?://\S+|www\.\S+|(\s)*\.(org|de|com)|^$').match(t):
    return False
      
 # special character,punctuation -> continue in this case 
 if re.compile(r'[^a-zA-Z0-9äöüÄÖÜ\s]').match(t):
    return False
 
 #is stopword
 if t in stop_words:
    return False 
 

 #filter out words that are for sure not english
 if (t in spellGerman):
     if(not t in spellEnglish): 
       return False
     
 
 #remove also special chars inside token
 t = re.sub(r'[^a-zA-Z0-9äöüÄÖÜ\s]', '', t)
      
 #convert pos tag for lemmatization
 ltag = getWordnetPos(tag)
 if ltag:
   t = lemmatizer.lemmatize(t, ltag)

 return t

In [5]:
#create the directory for documents and pictures 
def create_directories():
    base_dir = 'static'
    directories = ['documents', 'pictures']

    for directory in directories:
        dir_path = os.path.join(base_dir, directory)
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
            print(f"Verzeichnis '{dir_path}' wurde erstellt.")
        else:
            print(f"Verzeichnis '{dir_path}' existiert bereits.")


In [6]:
#### Indexing ####

#Add a document to the index. You need (at least) two parameters:
    #doc: The document to be indexed.
    #index: The location of the local index storing the discovered documents.
def index(doc, index):
 url = doc['url']
 indexPath = index

 #get index of index.json
 with open(indexPath, 'r', encoding='utf-8') as f:
     index = json.load(f)
     docID = len(index[1])
     print(docID)
     doc = doc['doc']
     f.close()


####  Text Preprocessing of the doc ###################

 #extract relevant text from raw html doc
 content = extract(doc)

 # proof if english -> break if not
 # test on first 200 Words (if available)
 splittedText = content.split()
 if len(splittedText) >= 200:
     testText = ' '.join(splittedText[:200])
 else:
     testText = ' '.join(splittedText[:len(splittedText)])
 if(detect(testText) != "en"):
      return [False, docID]


###storing unprocessed text content ##### -> store as file at end if no duplicate
 docContents={}
 soup = bs(doc, 'html.parser')
 
 title = re.sub(r'[\n\t]', '', soup.find('title').text) if soup.find('title') else 'No title'
 docContents['title'] = title
 meta_desc = soup.find('meta', attrs={'name': 'description'})

 ###if description available save description, else save first 40 words (or less if doc shorter) -> remove special characters from that ####
 #take document description if available and at least 10 words long and in english else take first 40 words of first sensful paragraph as preview
 if meta_desc and (len(meta_desc['content'].split()) >=10) and detect(meta_desc['content']) =="en":
     docContents['text'] = meta_desc['content'] 
 else:
    #find a paragraph and take first 40 of it. Else take arbitrary first 40
    for p in soup.findAll('p'):
           beginning = p.text.split()
           if len(beginning) >=40:
               splittedText = beginning
               break
    #splittedText = soup.find('p').text.split() if soup.find('p') else splittedText
    if len(splittedText) >= 40:
     docContents['text'] = re.sub(r'[^a-zA-Z0-9äöüÄÖÜ\s!"#$%&\'()*+,\-\./:;<=>?@[\\]^_`{|}~]', '', ' '.join(splittedText[:40]))
    else: 
     docContents['text'] = re.sub(r'[^a-zA-Z0-9äöüÄÖÜ\s!"#$%&\'()*+,\-\./:;<=>?@[\\]^_`{|}~]', '', ' '.join(splittedText[:len(splittedText)]))

    
    #append "..." if text not ending at "."
    if docContents['text'][-1] != ".":
        docContents['text'] += "..."
        
 docContents['text'] =  re.sub(r'[\n\t\r]', ' ', docContents['text'])

 #docContents['content'] = content
 # Find the first image
 first_image = soup.find('img')
 
 
 #convert to lowercase
 content = content.lower()

 #remove contractions:
 content = contractions.fix(content)
 

 #store words of the processed document
 processedDoc = []
 #tokenize, go through all words and do the steps for each at once
 for position, (t, tag) in enumerate(pos_tag(word_tokenize(content))):
    
      #break up hyphenated words:
       if '-' in t:
          t = t.split('-')
          firstTermPreprocessed = preprocess(t[0], tag)
          #if not uninformative
          if firstTermPreprocessed:
              processedDoc.append([firstTermPreprocessed, position])
          secondTermPreprocessed = preprocess(t[1], tag)
          if secondTermPreprocessed:
              processedDoc.append([secondTermPreprocessed, position])
       else:
           termPreprocessed = preprocess(t, tag)
           if termPreprocessed:
               processedDoc.append([termPreprocessed, position])

    
      
  ############# Near Duplicate checking ##############
  #near duplicate checking by means of the words in the processed document (processedDoc)

 docHash = computeHash([l[0] for l in processedDoc])
 # compare to other doc hashes
 if(check_simhash(docHash, index[2])):
     #break and dont index if is near duplicate
     return [False, 0]
 #if no duplicate save hash and insert terms in inverted index
 index[2].append(docHash)



############ Build up inverted index #####################
 for term, position in processedDoc:
     
      #entry in inverted index is: [docID, [occurence1, occurence2, ...]]
      #add on thy fly to inverted index
      #if word already in index add doc idto the words list
      if(term in index[0].keys()):
           #check if doc id already in list of that word (must be at the end)
           if index[0][term][len(index[0][term])-1][0] == docID:
                #if so append position there
                index[0][term][len(index[0][term])-1][1].append(position)
           #else add new list [docID, [position of t],skip pointer, tfidfval] for that word
           else:
               index[0][term].append([docID, [position], None])
               
      #if word not yet in index add "t: [[docID, [get position of t], tfidf weight for t in d, skip pointer (None) , tfidfval]]" to dict
      else:
           index[0][term] = [[docID, [position], None]]

 length = len(processedDoc)

 #add url , cluster of document (None so far) and length of preprocessed doc to list index[1] after indexing the doc
 index[1].append([url, None, length])

 #write changed index object to json
 with open(indexPath, 'w', encoding='utf-8') as f:
     json.dump(index, f, ensure_ascii=False)
 
 #save doc in documents ordner: name <docID>.json
 with open(os.path.join(os.getcwd(), "static", "documents", str(docID) + ".json"), 'w', encoding='utf-8') as f:
     json.dump(docContents, f, ensure_ascii=False)
 
 #save first image of page if found
 if first_image:
     img_url = first_image['src']
     # Convert relative URL to absolute URL
     absolute_image_url = urljoin(url, img_url)
     f_ext = os.path.splitext(absolute_image_url)[-1]
     image_fetched = requests.get(absolute_image_url)
     if image_fetched.status_code == 200:
       # Get the content of the image
        image_content = image_fetched.content
       
        #save img in pictures ordner: name <docID>
        with open(os.path.join(os.getcwd(), "static" ,"pictures", str(docID)) + f_ext, 'wb') as f:
           f.write(image_content)

 return [True, docID]

In [7]:
#### Clustering ####
#clusters the docs of the index and inserts the labels into the index (currently 15 clusters)
# cluster using kmeans clustering with tf-idf vectors
def cluster(index):
     print('start clustering')
     indexPath = index
     #get index of index.json
     f = open(indexPath, encoding='utf-8')
     index = json.load(f)
     #convert index to tf-idf vector representations for each document
     idx = index[0] 
     docs = index[1]
     #matrix to store vectors (rows: documents , cols: terms)
     tfIDFMatrix = np.zeros((len(docs), len(idx.keys())))
     print('buildMatrix')

     for t in range(len(idx.keys())):
         term = list(idx.keys())[t]
         for i in range(len(idx[list(idx.keys())[t]])):
             #term index: t, doc index: idx[t][i][0]

             docID = idx[term][i][0]
             
             #calculate tf (nr occurences of t in doc/ length of doc) * idf(log (#docs/ #docs containing t))
             # occurences of t in doc: len(idx[term][i][1])
             # length of doc: docs[docID][2]
             # # docs = len[docs]
             # #docs containing term = len(idx[term])
             tfValue = len(idx[term][i][1])/docs[docID][2]
             #idfValue = math.log(len(docs)/len(idx[term]))
             tfIDFMatrix[docID][t] = tfValue #* idfValue

             ## store idf value for that doc and term in index on the fly (for BM25 later on)
             #index[0][term][i][3] = idfValue
     
     print('endBuildmatrix')

     #also calculate avg doc length and append to index
     docLengths = [l[2] for l in index[1]]
     avgLength = sum(docLengths)/len(docLengths)
     index[3] = avgLength
      
     #place skip pointers once after whole index is built up
     #### rearange skip pointers ###########
     #delete current skip pointers for that posting list
     for term in index[0].keys():
        # sqare |p| evenly spaced per posting list (p: length of posting list of term t)
        p = len(index[0][term])
        #just if posting list has at least length 4
        if(p >= 4):
            nrPointers = math.sqrt(p)
            spaceBetween = math.floor(p/nrPointers)
            #current index in postingslist
            i = 0
            while(i + spaceBetween < p):
                #set skip pointer [idx to jump to in postings list, docID at that index]
                index[0][term][i][2] = [i + spaceBetween, index[0][term][i + spaceBetween][0]]
                i += spaceBetween

     #store new  index with tf idf values 
     #write changed index object to json
     with open(indexPath, 'w', encoding='utf-8') as f:
      json.dump(index, f, ensure_ascii=False)
     # Initialize KMeans clustering
     print('clustering')
     num_clusters = 30 # Example: Number of clusters
     kmeans = KMeans(n_clusters=num_clusters, random_state=42)

     # Fit KMeans model to the TF-IDF matrix
     kmeans.fit(tfIDFMatrix)

     docLabels = kmeans.labels_
     #centroids of each cluster:
     centroids = kmeans.cluster_centers_

     ###get term for each cluster #####
      
     #get term with highest tf-idf score per cluster centroid
     centroidTopics = []
     for centroid in centroids:

        ###get x terms with hightest tf-idf value
        # indices for words in descending order (regarding tf-idf val)
        sorted_indices = np.argsort(centroid)[::-1]
        #take top 50 words:
        top50Indices = sorted_indices[:50]
        #words of these indices
        top50Words = [list(idx.keys())[l] for l in top50Indices]
    

       ### take word with highest tf value in centroid as topic for each cluster ###
       # search for first sensful word (more than one char and not only number) and not tübingen
        topic = ' '
        for i in range(len(top50Words)):
         if len(top50Words[i])>1 and not top50Words[i].isnumeric() and top50Words[i] != "tübingen":
          topic = top50Words[i].capitalize()
          break
        #print("centroid words: " + str(top50Words))
        #print(topic)
        centroidTopics.append(topic)
     
     
     #insert labels in index
     for d in range(len(index[1])):
         index[1][d][1] = centroidTopics[docLabels[d]]
      
     print('endClustering')
        
      #write changed index object to json
     with open(indexPath, 'w', encoding='utf-8') as f:
      json.dump(index, f, ensure_ascii=False)
     
     return True

In [12]:
#### Crawling ####
#Crawl the web. You need (at least) two parameters:
#frontier: The frontier of known URLs to crawl. You will initially populate this with your seed set of URLs and later maintain all discovered (but not yet crawled) URLs here.
#index: The location of the local index storing the discovered documents. 
STORAGE_LOC = "index.json"
FRONTIER_LOC = "frontier.json"
MAX_DOCS = 800
reshuffle_at = 11


def crawl(frontier, indexPath):
    f = open(frontier, encoding='utf-8')
    frontierObj = json.load(f)
    frontier = frontierObj["frontier"]
    links_seen = frontierObj["linksSeen"]
    f.close()

    
    #get  first document of frontier while frontier not empty
    while len(frontier) != 0:
        with stopit.ThreadingTimeout(20) as context_manager:
            link = frontier.pop(0)
            print(link)
            try:
                base_link = urlparse(link).netloc
                
                # check if we are allowed to access the website
                robots_file_loc = "http://" + base_link + "/robots.txt"
                session = requests.Session()
                retry = Retry(total=5, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])
                adapter = HTTPAdapter(max_retries=retry)
                session.mount('https://', adapter)
                robots_file = session.get(robots_file_loc, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"})
                session.close()
                if robots_file.ok:
                    robot_parser = rp()
                    robot_parser.set_url(robots_file_loc)
                    robot_parser.read()
                    if not robot_parser.can_fetch("*", link):
                        continue
                elif robots_file.status_code != 404:
                    continue
                
                session = requests.Session()
                retry = Retry(total=5, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])
                adapter = HTTPAdapter(max_retries=retry)
                session.mount('https://', adapter)
                document = session.get(link, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}).text
                session.close()
                html5_doc = html5lib.parse(document)
                soup = bs(document, "html.parser")

                # save and use document if it's not a duplicate
                for script in soup(["script", "style"]):
                    script.extract() 

                # check if document is relevant
                doc_lang = html5_doc.get("lang")
                if doc_lang == None:
                    doc_lang = html5_doc.get("xml:lang")
                if doc_lang == None:
                    continue
                relevant = soup.find("body").text.find("Tübingen") > -1 # type: ignore
                if (doc_lang is not None and doc_lang.count("en") > 0 and relevant): # type: ignore
                    # process doc to right format and index
                    doc = {"url": link, "doc": document }
                    # TODO: index document
                    duplicate, docAmount = index(doc, indexPath)
                    if duplicate == False:
                        print("Duplicate/ Not English")
                        continue
                    # get all links from document and save to frontier if not seen yet
                    for a in soup.find_all('a'):
                        if (a.get('href')):
                            l = a.get('href')
                            if l.startswith('#'):	
                                continue
                            if urlparse(l).netloc == '':
                                l = base_link + l
                            if urlparse(l).scheme == '':
                                l = urlparse(link).scheme + '://' + l
                            if l not in links_seen:
                                frontier.append(l)
                                links_seen.append(l)
                    
                    if docAmount >= MAX_DOCS:
                        break
                    global reshuffle_at
                    if docAmount == reshuffle_at:
                        random.shuffle(frontier)
                        reshuffle_at = len(links_seen)
                    
                #save frontier and links seen
                with open(FRONTIER_LOC, 'w', encoding='utf-8') as f:
                    json.dump({"frontier": frontier, "linksSeen": links_seen}, f, ensure_ascii=False)
                   

                    
            except Exception as err:
                if context_manager.state == context_manager.TIMED_OUT:
                    print("Timed out at link: ", link)
                else:
                    print("Exception occured at link: ", link, ". Description: ", err)



## Reset before crawling
Execute the following cell in case you want to restart the whole crawling process and don't just continue where you stopped

In [16]:
#### Execution (crawling, indexing, clustering) ####
## delete content of documents and pictures folder if there is some
# delete index
# also set frontier/ links seen back to initial frontier/links seen
# so the process can be started again without resetting everything by hand
def reset():
    docs = [os.path.join(os.getcwd(), "static", "documents", f) for f in os.listdir(os.path.join(os.getcwd(), "static",  "documents"))]
    for f in docs:
        os.remove(f)

    images = [os.path.join(os.getcwd(), "static", "pictures", f) for f in os.listdir(os.path.join(os.getcwd(), "static", "pictures"))]
    for i in images:
        os.remove(i)

    #write [{}, [], [], None] in index.json 
    json_data = json.dumps([{}, [], [], None])
    with open(STORAGE_LOC, 'w') as json_file:
        json_file.write(json_data)
        json_file.close()
    
    #write initial frontier to frontier.json
    initalLinks = [    
            "https://www.tuebingen.de/en/",  
            "https://is.mpg.de/",  
            "https://www.iwm-tuebingen.de/www/en/index.html",
            "https://kunsthalle-tuebingen.de/en/",
            "https://www.dzne.de/en/about-us/sites/tuebingen/",
            "https://www.tuebingen-info.de/en",
            "https://uni-tuebingen.de/en/",
            "https://www.eventbrite.com/d/germany--tuebingen/events/",  
            "https://www.opentable.com/s/?city=Tuebingen&country=Germany",  
            "https://www.yelp.com/search?find_desc=Restaurants&find_loc=T%C3%BCbingen%2C+Germany", 
            "https://en.wikipedia.org/wiki/T%C3%BCbingen", 
            "https://www.tagblatt.de/",
            "https://www.nomadicmatt.com/travel-blogs/tubingen/",
            "https://www.life-in-germany.net/tubingen/",
            "https://www.reddit.com/r/germany/",
            "https://tuebingenresearchcampus.com/en/tuebingen",
            "https://tunewsinternational.com/category/news-in-english/",
            "https://www.germany.travel/en/cities-culture/tuebingen.html",
            "https://www.opentable.com/food-near-me/stadt-tubingen-germany",
            "https://historicgermany.travel/historic-germany/tubingen/"
            "https://en.wikipedia.org/wiki/University_of_T%C3%BCbingen"]
    
    with open(FRONTIER_LOC, 'w') as frontier_file:
        frontier_file.write(json.dumps({"frontier":initalLinks, "linksSeen": initalLinks}))
        frontier_file.close()

reset()

## Crawl, index

In [8]:
### crawl ####
create_directories()
crawl(FRONTIER_LOC, STORAGE_LOC)

Verzeichnis 'static\documents' existiert bereits.
Verzeichnis 'static\pictures' existiert bereits.
https://en.wikipedia.org/wiki/Academy_of_Fine_Arts,_Karlsruhe
https://uni-tuebingen.de/en/alumni/research-alumni/
473
https://www.germany.travel/es/ciudades-y-cultura/tubinga.html
https://uni-tuebingen.de/en/faculties/center-for-islamic-theology/center/
474
https://uni-tuebingen.de/en/fakultaeten/juristische-fakultaet/lehrstuehle-und-personen/
475
Duplicate/ Not English
https://uni-tuebingen.de/en/international/studieren-im-ausland/vorbereitung/
475
Duplicate/ Not English
https://en.wikipedia.org/wiki/Template:German_Excellence_Universities
475
https://no.wikipedia.org/wiki/Eberhard-Karls-Universit%C3%A4t_T%C3%BCbingen
https://tuebingenresearchcampus.com/#end
476
Duplicate/ Not English
https://nl.wikipedia.org/wiki/Eberhard-Karls-Universiteit
https://en.wikipedia.org/wiki/Wayback_Machine
https://www.germany.travel/en/nature-outdoor-activities/the-world-of-german-animals-more-than-fox-and-

c:\Users\Pascal\AppData\Local\Programs\Python\Python310\lib\html\parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


https://uni-tuebingen.deen/324
Exception occured at link:  https://uni-tuebingen.deen/324 . Description:  HTTPConnectionPool(host='uni-tuebingen.deen', port=80): Max retries exceeded with url: /robots.txt (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001856C378B20>: Failed to resolve 'uni-tuebingen.deen' ([Errno 11001] getaddrinfo failed)"))
https://en.wikipedia.org/w/index.php?title=Special:CreateAccount&returnto=G%C3%BCnther+Oettinger
https://en.wikipedia.org/w/index.php?title=Special:UserLogin&returnto=G%C3%BCnther+Oettinger
https://als.wikipedia.org/wiki/G%C3%BCnther_Oettinger
https://ar.wikipedia.org/wiki/%D8%BA%D9%88%D9%86%D8%AA%D8%B1_%D8%A3%D9%88%D8%AA%D9%8A%D9%86%D8%BA%D8%B1
https://bg.wikipedia.org/wiki/%D0%93%D1%8E%D0%BD%D1%82%D0%B5%D1%80_%D0%99%D0%BE%D1%82%D0%B8%D0%BD%D0%B3%D0%B5%D1%80
https://ca.wikipedia.org/wiki/G%C3%BCnther_Oettinger
https://cs.wikipedia.org/wiki/G%C3%BCnther_Oettinger
https://da.wikipedia.org/wiki/G%C3%BCnther_Oetting

## Cluster, add additional infos to index

In [10]:
#after crawling cluster and add remaining infos to index
cluster(STORAGE_LOC)

start clustering
buildMatrix
endBuildmatrix
clustering


c:\Users\Pascal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Pascal\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pascal\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Develop

centroid words: ['diversity', 'auditing', 'fullest', 'gender', 'allows', 'vielfalt', 'stifterverband', 'mediates', 'office', 'refine', 'gestalt', 'certify', 'disadvantage', 'sexual', 'embody', 'port', 'workplace', 'ethnic', 'regardless', 'audit', 'discrimination', 'culture', 'disability', 'valid', 'subsequent', 'implementation', 'objective', 'equality', 'mission', 'identity', 'behind', 'initial', 'coordinate', 'certificate', 'origin', 'statement', 'background', 'review', 'aspect', 'phase', 'based', 'goal', 'concept', 'contribute', 'religious', 'enable', 'cover', 'responsible', 'relation', 'advice']
topic: Culture


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


centroid words: ['category', 'page', 'reflect', 'total', 'recent', 'subcategories', 'appearance', 'change', 'following', 'list', 'wikimedia', '7', 'may', 'establishment', 'attraction', 'tourist', 'college', 'category1477', 'relate', 'follow', 'common', 'category1470s', 'st', 'building', 'relevant', 'choose', 'holy', '1470s', '18', 'tübingen', '5', 'empire', 'structure', 'f', 'medium', '530', 'roman', 'grammar', 'württemberg', 'way', '49', 'enrollment', '11', '1477', 'p', '15th', 'l', 'europe', 'educational', 'first']
topic: Religion


...

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


centroid words: ['candle', 'light', 'caliph', 'lamp', 'odour', 'oil', 'rite', 'fat', 'use', 'scent', 'ambergris', 'umayyad', 'pharaonic', 'ibn', 'malik', 'olive', 'lantern', 'abd', 'wedding', 'occasion', 'clay', 'procession', 'al', 'animal', 'roman', 'wealthy', 'wood', 'ritual', 'metal', 'electricity', 'burn', 'perhaps', 'war', 'muslim', 'ago', 'people', 'church', 'yazid', 'wax', 'beeswax', '2130', 'camphor', 'fatimid', 'shama', 'tun23112904', 'kerzen', 'jahrhunderten', 'kreuzer', '002233', '002235']
topic: Culture


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


centroid words: ['982', 'uk', 'e', 'news', 'integreat', '992', 'everyday', '10', '2023', 'leisure', 'residence', 'corona', '2022', 'jul', 'jun', 'apr', 'mar', 'feb', '2024', 'oct', 'aug', 'learning', 'events', 'living', '2', '6', 'health', 'refugee', '9', '5', 'life', 'language', '8', '7', '2021', 'dec', '0', '1', 'nov', '3', '2020', '4', 'article', 'sep', 'jan', 'impfnews', '892', 'region', 'main', 'event']
topic: Engineering
endClustering


True

# 2. Query Processing 
Process a textual query and return the 100 most relevant documents from your index. Please incorporate **at least one retrieval model innovation** that goes beyond BM25 or TF-IDF. Please allow for queries to be entered either individually in an interactive user interface (see also #3 below), or via a batch file containing multiple queries at once. The batch file will be formatted to have one query per line, listing the query number, and query text as tab-separated entries. An example of the batch file for the first two queries looks like this:

```
1   tübingen attractions
2   food and drinks
```

In [ ]:
# Process query
# remove stop words, lemmatize, etc.
# Todo: Funktion von Alex nutzen?
def prepare_query(query):
	"""
	Function takes the query and processes it for further calculations the same way as it is done for documents for indexing.
	Function removes stop words and dublicates, tokenizes and lematizes the query  
	Args:
		query: our initial query given by the user

	Returns: 
		query_tokenized: list of the processed query items

	"""
	query_tokenized= []
	stop_words = set(stopwords.words('english'))
	lemmatizer = WordNetLemmatizer()
	query_lower = query.lower()
	for term, tag in pos_tag(word_tokenize(query_lower)):
		if re.compile(r'[^a-zA-Z0-9äöüÄÖÜ\s]').match(term):
			continue
		if term in stop_words:
			continue

		# split words with "-"
		if '-' in term:
			t = term.split('-')
			for i, tag in pos_tag(t):
				ltag = tag[0].lower()
				if ltag in ['a', 'r', 'n', 'v']:
					i = lemmatizer.lemmatize(i, ltag)	
					if i not in query_tokenized:
						query_tokenized.append(i)
		else: # words without "-"
			ltag = tag[0].lower()
			if ltag in ['a', 'r', 'n', 'v']:
				term = lemmatizer.lemmatize(term, ltag)	
				if term not in query_tokenized:
					query_tokenized.append(term)
	return query_tokenized

#get all the documents that include all the queryterms from our index 
def get_documents_from_index(query_tokenized, index):
	"""
	Function takes the query and the index, searches for documents in index which contain all the query-terms- 
	Args:
		query_tokenized: our processed query 
		index: index containing all terms and matching documents

	Returns: 
		documents: list of all documents matching our query

	"""
	documents = []
	first_document = True
	second_document = False

	for term in query_tokenized:
		# first term in query --> get all documents amtching this term from index
		if term in index.keys() and first_document:   		
			first_document = False	
			documents = index[term]
			second_document = True
		# check second term. get documents from first iteration which have this term as well. use skip pointers for both lists 
		elif term in index.keys() and second_document: 
			second_document = False
			matches = []
			new_term_list = index[term]
			i = 0
			j = 0
			while i < len(documents) and j < len(new_term_list):  
				if documents[i][0] == new_term_list[j][0]:
					matches.append(documents[i])							# append entry including skip pointers for future iterations 
					i += 1
					j += 1
				elif documents[i][0] < new_term_list[j][0]:                 # A < B
					if documents[i][2]:                                     # If there is a skip pointer
						if documents[i][2][1] <= new_term_list[j][0]:       # Take it if it does not carry you beyond the id pointed to by B
							i = documents[i][2][0]
						else:                                       		# Otherwise increase the pointer by 1
							i += 1
					else:
						i += 1
				elif new_term_list[j][2]:                                   # If there is a skip pointer
					if new_term_list[j][2][1] <= documents[i][0]:           # Take it if it does not carry you beyond the id pointed to by A
						j = new_term_list[j][2][0]
					else:
						j += 1                                     			# Otherwise increase the pointer by 1
				else:
					j += 1 
			documents = matches		
			
		# check current term. get documents from previous iteration which have this term as well. use skip pointers only for list in index.
		elif term in index.keys():
			matches = []
			new_term_list = index[term]
			i = 0
			j = 0
			while i < len(documents) and j < len(new_term_list):  
				if documents[i][0] == new_term_list[j][0]:
					matches.append(documents[i])							# append entry including skip pointers for future iterations 
					i += 1
					j += 1
				elif documents[i][0] < new_term_list[j][0]:    				# A < B
					i += 1													# This time no skip pointers, because they aren't correct anymore

				elif new_term_list[j][2]:                                   # If there is a skip pointer
					if new_term_list[j][2][1] <= documents[i][0]:           # Take it if it does not carry you beyond the id pointed to by A
						j = new_term_list[j][2][0]
					else:
						j += 1                                     			# Otherwise increase the pointer by 1
				else:
					j += 1 
			documents = matches		
		else:
			return []														# query term wasn't found, no document can satisfy query

	# get rid of skip pointers:
	tmp = []
	for element in documents:
		tmp.append(element[0])
	documents = tmp

	return documents

# calculate bm25 score for each document
def bm25(document, query, index, k, b, avg_length, document_length, doc_num) :
	"""
	Function calculates the bm25 score for a document-query pair.  
	Args:
		document: a document matching our query
		query: our initial query given by the user
		index: index containing all terms and matching documents
		k: parameter for optimization
		b: parameter for optimization
		avg_length: average length of all documents in our index
		document_length: length of the document
		doc_num: total number of documents in our index

	Returns: 
		score: bm25 score for our document-query pair

	"""
	score = 0
	
	#calculate score:
	for term in query:
		tf = 0
		idf = 0
		# get tf from index
		for doc in index[term]:
			if doc[0] == document:
				tf = len(doc[1])

		#calculate idf:
		idf = math.log(doc_num/len(index[term]))

		numerator = tf *(k+1)
		denominator = tf + (k*(1 - b + (b* (document_length/avg_length))))
		#add score of query-term to score:
		score += idf * (numerator/denominator)

	return score


def get_distance(document, query, index):
	"""
	Function minimal distance between two adjacent query terms in one document.
	Args:
		document: a document matching our query
		query: our initial query given by the user
		index: index containing all terms and matching documents

	Returns: 
		avg_distance: average minimal distance between adjacent query terms 
		max_min_distance: the biggest minimal distance
	"""
	sum_of_distances = 0
	avg_distance = 0
	min_distance = math.inf 							# minimal distance between two terms of a query in the document.
	max_min_distance = 0 								# the maximum minimal distance
	query_length = len(query)
	num_comparisons = query_length - 1 					# number of comparisons, needed for normalization
	count = 0
	for i in range(query_length -1): 					# term in query
		min_distance = math.inf 						#minimal distance between two terms of a query in the document.
		count +=1
		term_list_1 = []
		term_list_2 = []

		for doc in index[query[i]]:
			if doc[0] == document:
				term_list_1 = doc[1] 					# get positional lists for document and term one
		for doc in index[query[i+1]]:
			if doc[0] == document:
				term_list_2 = doc[1] 					# get positional lists for document and term two
		# find smallest distance between terms:
		for pos1 in term_list_1:
			for pos2 in term_list_2:
				distance = abs(pos1 - pos2)
				if distance < min_distance:
					min_distance = distance
		sum_of_distances += min_distance
		if min_distance > max_min_distance: 			# get maximum minimal distance, used for normalitazion
			max_min_distance = min_distance

	# get avg_distance between two terms
	avg_distance = sum_of_distances/num_comparisons 		
	return avg_distance, max_min_distance

# work in progress
def position_score(documents, query, index):
	"""
	Function calculates posiion score for a document-query pair. If terms in documents are near each other, a higher score is given.
	Args:
		document: a document matching our query
		query: our initial query given by the user
		index: index containing all terms and matching documents

	Returns: 
		position_score_list: list with score for each document, based on how close terms form query are in the document. Score is value between 0 and 1

	"""
	distance_score_list = []
	position_score_list = []
	max_dist = 0
	for document in documents:
		normalized_distance, max_min_distance =	get_distance(document, query, index)
		if max_min_distance > max_dist:
			max_dist = max_min_distance
		distance_score_list.append(normalized_distance)

	for distance in distance_score_list:
		position_score_list.append(1 - (distance/max_dist)) # calculate score

	return position_score_list


def ranking_sort_helper(document_and_score):
	"""
	Function returning the score of a given document, used for sorting the ranked list.
	Args:
		document_and_score: list entry with document and score pair

	Returns: 
		score value
	"""
	return document_and_score[1]

def format_helper_ui(ranking, url_cluster):
	"""
	Function formating the ranked list for ui-output. Adding necessary information.
	Args:
		ranking: ranked list with documend-id and score pairs 
		url_cluster: List with document-id, associated URL and cluster.
	Returns: 
		result: ranked list with all the necessary information

	"""
	result = []
	for position, entry in enumerate(ranking):
		URL = url_cluster[entry[0]][0]
		cluster = url_cluster[entry[0]][1]
		result.append([position + 1, URL, entry[0], cluster])
	return result

def format_helper_file(ranking, url_cluster):
	"""
	Function formating the ranked list for text-file-output. Adding necessary information.
	Args:
		ranking: ranked list with documend-id and score pairs 
		url_cluster: List with document-id, associated URL and cluster.
	Returns: 
		result: ranked list with all the necessary information

	"""
	result = []
	for entry in ranking:
		URL = url_cluster[entry[0]][0]
		result.append([URL, entry[1]])
	return result 

#Measure the average relevance of a (partial) result list. 
def measure_avg_relevance(ranking):
	"""
	Function calculating the average relevance of a result list
	Args:
		ranking: ranked list with documend-id and score pairs 
	Returns: 
		average relevance score for this list 

	"""
	relevance = 0.0

	for doc in ranking:
		relevance += doc[1]
	return relevance/len(ranking)

#Measure the diversity of a (partial) result list. Diversity is meassured by the difference of sim hash values of those documents
def measure_diversity(ranking, hashes):
	"""
	Function calculating the diversity of a result list
	Args:
		ranking: ranked list with documend-id and score pairs 
		hashes: list of sim hashes, used for calculating similarity
	Returns: 
		diversity: diversity score of list

	"""
	diversity = 0.0
	similarity = 0
	hash_length = len(hashes[0])
	for i in range(hash_length):
		is_equal = True
		for j in range(len(ranking) - 1): 								# loop over each bit in the sim hash
			if hashes[ranking[j][0]][i] != hashes[ranking[j+1][0]][i]:  # check if hash of two following documents have the same value at bit j of hash.  
				is_equal = False										# if two documents do not have the same bit they aren't similar.
				break													# no need for further comparison
		if is_equal:
			similarity += 1	
	diversity = (hash_length - similarity) / (hash_length)				# normalize to values between 0 and 1
	return diversity

# Todo: überprüfen und überarbeiten
def diversify(ranking, hashes, num_entries, beta):
	"""
	Function diversifies a ranked list
	Args:
		ranking: ranked list with documend-id and score pairs 
		hashes: list of sim hashes, used for calculating similarity
		num_entries: number of entries which should be in the diversified list
		beta: parameter which determines ratio between relevance and diversity in   
		 
	Returns: 
		reranked: reranked result list with document score pairs

	"""
	reranked = []
	if len(ranking) < num_entries:	# prevent index out of bounce
		num_entries = len(ranking)

	reranked.append(ranking[0])
	del ranking[0]

	while len(reranked) < num_entries:		
		greedy = 0
		index = 0
		for doc in ranking:
			pre_list = []
			pre_list += reranked
			pre_list.append(doc)
			
			relevance = measure_avg_relevance(pre_list)
			diversity = measure_diversity(pre_list, hashes)
			weighted_mix = beta * relevance + (1-beta) * diversity 
			if(weighted_mix > greedy):
				greedy = weighted_mix
				index = ranking.index(doc)
			
		reranked.append(ranking[index])
		del ranking[index]		

	return reranked

#Retrieve documents relevant to a query. You need (at least) two parameters:
    #query: The user's search query
    #index: The location of the local index storing the discovered documents.
def retrieve(query, index, is_batch=False ):
	"""
	Function takes a query and index. Ranks documents matching the query for relevance.  
	Args:
		query: our initial query given by the user
		index: index containing all terms and matching documents
		is_batch: boolean, if true, queries are from text-file, return format differs from UI-query, default is false

	Returns: 
		result_list: list of ranked documents

	"""
	ranking = []
	result_list = []
	alpha = 0.7 # alpha value used for weighting bm_25 score, should be between 0 and 1
	beta = 0.8 # beta value used for weighting relevance score and diversity. Beta should be between 0 and 1
	reranked = []

	# get processed query
	query_tokenized= prepare_query(query)		
	#print('query tokenized:', query_tokenized,'\n')

	#get dictionary from index
	document_index = index[0]
	#get avg document length:
	avg_length = index[3]
	# get number of documents in index:
	num_doc = len(index[1])
	hashes = index[2]

	# get all the documents, that include all the terms from our query
	documents = get_documents_from_index(query_tokenized, document_index)	
	#print('erhaltene Dokumente', documents,'\n')
	
	# get position scores
	# get them only if there are more than one term in the query, otherwise can't be calculated.
	if len(query_tokenized) > 1:
		pos_scores = position_score(documents, query_tokenized, document_index)
	else:
		pos_scores = [1 for _ in range(len(documents))]

	max_bm25_score = 0
	bm25_score_list = []
	# calculate bm 25 score for each document:
	for document in documents:
		document_length = index[1][document][2]

		# Todo: werte für k & b optimieren
		# get Bm25 score
		bm25_score = bm25(document, query_tokenized, document_index ,1.5, 0.75, avg_length, document_length, num_doc)
		bm25_score_list.append([document, bm25_score])
		if bm25_score > max_bm25_score:
			max_bm25_score = bm25_score

	# calculate relevance score for each document 
	for i, entry in enumerate(bm25_score_list):
		# get position score
		pos_score = pos_scores[i]
		bm25_score_normalized = entry[1]/max_bm25_score
		#calculate score based on bm25 score and position score
		score = (alpha * bm25_score_normalized + (1-alpha) * pos_score)

		ranking.append([entry[0], score])

	
	# Sort ranking based on relevance score 
	ranking.sort(key=ranking_sort_helper, reverse=True)

	print('ranking', ranking)

	# diversify result list
	if len(ranking) >=2: #only rerank, if enough items in list 
		reranked = diversify(copy.copy(ranking), hashes, 99, beta)
	else: 
		reranked = ranking
	print('reranked', reranked)

	#format output
	if is_batch:
		#get ranking into right format for txt-file:
		result_list = format_helper_file(reranked[:99], index[1])
	else:
		#get ranking into right format for srp visualization:
		result_list = format_helper_ui(reranked[:99], index[1])

	return result_list

with open('index.json', encoding='utf-8') as file:
    index = json.load(file)

retrieve('study programs', index, True)

[['https://uni-tuebingen.de/en/study/finding-a-course/degree-programs-available/',
  0.9998484082870136],
 ['https://uni-tuebingen.de/en/facilities/administration/iii-academic-affairs/',
  0.9256309163984865],
 ['https://uni-tuebingen.de/en/study/application-and-enrollment/enrolling-at-the-university-of-tuebingen/',
  0.9609015272580705],
 ['https://uni-tuebingen.de/en/faculties/faculty-of-science/departments/',
  0.9661543975225613],
 ['https://uni-tuebingen.de/en/study/finding-a-course/', 0.9746233414433663],
 ['https://uni-tuebingen.de/en/international/university/branch-offices-and-research-stations/',
  0.9770758112322045],
 ['https://uni-tuebingen.de/en/study/organizing-your-studies/orientation/',
  0.995483950878667],
 ['https://uni-tuebingen.de/en/international/studying-abroad/ways-to-go-abroad/',
  0.9772115638941964],
 ['https://en.wikipedia.org/wiki/Karlshochschule_International_University',
  0.9289135817435132],
 ['https://uni-tuebingen.de/en/international/study-in-tuebinge

# 3. Search Result Presentation
Once you have a result set, we want to return it to the searcher in two ways: a) in an interactive user interface. For this user interface, please think of **at least one innovation** that goes beyond the traditional 10-blue-links interface that most commercial search engines employ. b) as a text file used for batch performance evaluation. The text file should be formatted to produce one ranked result per line, listing the query number, rank position, document URL and relevance score as tab-separated entries. An example of the first three lines of such a text file looks like this:

```
1   1   https://www.tuebingen.de/en/3521.html   0.725
1   2   https://www.komoot.com/guide/355570/castles-in-tuebingen-district   0.671
1   3   https://www.unimuseum.uni-tuebingen.de/en/museum-at-hohentuebingen-castle   0.529
...
1   100 https://www.tuebingen.de/en/3536.html   0.178
2   1   https://www.tuebingen.de/en/3773.html   0.956
2   2   https://www.tuebingen.de/en/4456.html   0.797
...
```

In [60]:
#Produce a text file with 100 results per query in the format specified above.
def load_queries(query_file_path):
    queries = []
    with open(query_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                query_num = int(parts[0])
                query_desc = parts[1]
                queries.append((query_num, query_desc))
    return queries

def batch(query_file_path, output_file_path):
    queries = load_queries(query_file_path)
    
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for query_num, query_desc in queries:
            results = retrieve(query_desc, index, True)
            for rank, result in enumerate(results, start=1):
                result_line = f"{query_num}\t{rank}\t{result[0]}\t{result[1]}"
                output_file.write(result_line + '\n')

query_file_path = 'queries.txt'
output_file_path = 'batch_results_Sysala_Hirsch_Wenninger_Moser.txt'
batch(query_file_path, output_file_path)

[[258, 2.011638868341603], [398, 0.5759302956771458], [404, 1.8506832004081413], [497, 1.7168833469923934], [577, 9.281175077935094], [591, 4.925683905617056], [603, 2.362320971830445], [676, 2.9233953279139757], [766, 1.3295359400391074]]
ranking [[577, 0.99995770477936], [591, 0.6714600959075161], [676, 0.5204445194721249], [603, 0.4781274587593932], [258, 0.4516784940129158], [404, 0.4395389786382646], [497, 0.4294475951006682], [766, 0.4002332787462757], [398, 0.3433952225062313]]
reranked [[577, 0.99995770477936], [591, 0.6714600959075161], [676, 0.5204445194721249], [603, 0.4781274587593932], [258, 0.4516784940129158], [404, 0.4395389786382646], [497, 0.4294475951006682], [766, 0.4002332787462757], [398, 0.3433952225062313]]
[[52, 4.9641743202482225], [95, 6.2067041631128195], [129, 6.349935016238323], [394, 3.365389070058475], [480, 11.980472080982636], [498, 10.987793301503654], [567, 3.1119767327760695], [673, 2.9844376752936617]]
ranking [[480, 0.9997518610421836], [498, 0.94

In [ ]:
#Implement an interactive user interface for part a of this exercise.

app = Flask(__name__)

# Logging configure
logging.basicConfig(level=logging.DEBUG)

def load_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        logging.error(f"Error loading JSON from {file_path}: {e}")
    return {}

# Get the Image of each document
def find_image(doc_number):
    supported_formats = ['.svg', '.png', '.jpg', '.jpeg']
    for fmt in supported_formats:
        img_path = f'static/pictures/{doc_number}{fmt}'
        if os.path.exists(img_path):
            return img_path
    return 'static/pictures/default_picture.jpg'

# Path categories
categories = load_json_file('static/categories.json')
document_index = load_json_file('index.json')

# Get title and description
def get_info(doc_number):
    json_path = f'static/documents/{doc_number}.json'
    data = load_json_file(json_path)
    title = data.get('title', 'No title')
    description = data.get('text', 'No description')
    image_url = find_image(doc_number)
    return title, description, image_url

@app.route('/')
def index():
    return render_template('index.html', categories=categories)

@app.route('/search', methods=['GET', 'POST'])
def search():
    query = request.args.get('query') if request.method == 'GET' else request.form['query']
    print(query)
    ranklist = retrieve(query, document_index)
    #ranklist = ranklist[:10]
    print('list', ranklist)
    selected_filter = request.args.get('filter') if request.method == 'GET' else request.form.get('filter')
    filter_count = Counter([result[3] for result in ranklist])
    top_filters = [word for word, count in filter_count.most_common(5)]

    results = []
    for result in ranklist:
            if selected_filter and result[3] != selected_filter:
                continue
            doc_number = result[2]
            title, description, image_url = get_info(doc_number)
            results.append({'url': result[1], 'title': title, 'snippet': description, 'image': image_url})

    return render_template('results.html', query=query, results=results, top_filters=top_filters)


@app.route('/filter', methods=['POST'])
def filter():
    selected_filter = request.form['filter']
    query = request.form['query']
    return redirect(url_for('search', query=query, filter=selected_filter))

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


University of Tübingen
maxdist 867
distance [3.0, 2.0, 2.0, 10.0, 10.0, 14.0, 4.0, 2.0, 2.0, 14.0, 3.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 1.0, 2.0, 7.0, 2.0, 65.0, 2.0, 2.0, 7.0, 56.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 117.0, 2.0, 1.0, 2.0, 2.0, 2.0, 10.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 867.0, 2.0, 1.0, 3.0, 2.0, 7.0, 2.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 63.0, 2.0, 2.0, 3.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 26.0, 3.0, 2.0, 2.0, 2.0, 2.0, 60.0, 2.0, 101.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 102.0, 2.0, 2.0, 2.0, 2.0, 4.0, 7.0, 2.0, 2.0, 532.0, 2.0, 2.0, 2.0, 7.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 11.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 257.0, 1.0, 2.0, 2.0, 2.0, 8.0, 1.0, 7.0, 2.0, 101.0, 2.0, 1.0, 29.0, 2.0, 27.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 7.0, 41.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.

INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:11] "POST /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:11] "GET /static/styles.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:11] "GET /static/pictures/567.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:11] "GET /static/pictures/175.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:11] "GET /static/pictures/506.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:11] "GET /static/pictures/248.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:11] "GET /static/pictures/672.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:11] "GET /static/pictures/543.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:11] "GET /static/pictures/416.svg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:11] "GET /static/pictures/317.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:11] "

list [[1, 'https://en.wikipedia.org/wiki/University_of_T%C3%BCbingen', 6, 'University'], [2, 'https://uni-tuebingen.de/en/study/student-life/clubs-and-societies/', 331, 'University'], [3, 'https://uni-tuebingen.de/en/research/support/good-scientific-practice/', 653, 'University'], [4, 'https://en.wikipedia.org/wiki/File:University_of_T%C3%BCbingen_logo.svg', 431, 'University'], [5, 'https://uni-tuebingen.de/en/research/core-research/collaborative-research-centers/', 634, 'University'], [6, 'https://en.wikipedia.org/wiki/Faculty_of_Roman-Catholic_Theology,_University_of_T%C3%BCbingen', 112, 'University'], [7, 'https://uni-tuebingen.de/en/research/core-research/transregional-collaborative-research-centers-crc-trrs/', 462, 'University'], [8, 'https://uni-tuebingen.de/en/international/study-in-tuebingen/summer-courses-and-short-term-programs/', 544, 'University'], [9, 'https://uni-tuebingen.de/en/international/university/solidarity-with-ukraine/', 455, 'University'], [10, 'https://uni-tueb

...

INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:28] "POST /filter HTTP/1.1" 302 -


Libraries
[[6, 2.3817856352496043], [12, 2.3023555286739272], [22, 1.2597840821678152], [24, 2.832007877596222], [36, 1.7657837064138535], [39, 2.5017716684963496], [40, 2.5174322046210014], [44, 2.2084021946922885], [52, 1.50251320611492], [53, 2.0330930665167513], [61, 0.5769457684874547], [62, 1.9072173146840192], [65, 1.8940215615483018], [70, 2.552994180420866], [74, 2.5584531360190836], [77, 1.568978178517666], [87, 2.3088521506284], [91, 0.45194115903967413], [95, 2.6938810803297644], [96, 2.9410068728735297], [98, 2.343586075312382], [105, 1.9156779632395633], [109, 0.9965730255266567], [116, 1.4181072764474052], [118, 3.0876703792186477], [119, 2.658560595788967], [129, 1.8894871184994708], [133, 2.7866045265282957], [134, 0.9101727411521516], [136, 2.4417779495641305], [146, 2.003017368959868], [147, 2.7511618969419267], [149, 1.0470215150611568], [155, 1.7216118684622814], [157, 2.566253327830377], [165, 2.3386088050418326], [167, 1.8941158207972686], [185, 2.919252754179683

INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:28] "GET /search?query=Libraries&filter=University HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:28] "GET /static/styles.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:28] "GET /static/pictures/763.svg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:28] "GET /static/pictures/758.svg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:28] "GET /static/pictures/730.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:28] "GET /static/pictures/756.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:28] "GET /static/pictures/700.svg HTTP/1.1" 304 -


list [[1, 'https://uni-tuebingen.de/en/facilities/university-library/about-us/directions/', 763, 'University'], [2, 'https://uni-tuebingen.de/en/einrichtungen/universitaetsbibliothek/ueber-uns/fachbibliotheken/', 758, 'University'], [3, 'https://uni-tuebingen.de/en/facilities/university-library/publishing-research/publishing/framework-agreements-with-publishers/', 730, 'University'], [4, 'https://uni-tuebingen.de/en/facilities/university-library/about-us/training/', 756, 'University'], [5, 'https://uni-tuebingen.de/en/facilities/university-library/searching-borrowing/library-card-loans/loans/', 700, 'University'], [6, 'https://uni-tuebingen.de/en/facilities/university-library/about-us/a-profile-of-the-library/branch-library-schloss-nord/', 748, 'University'], [7, 'https://uni-tuebingen.de/en/facilities/university-library/searching-borrowing/library-card-loans/library-card/', 699, 'University'], [8, 'https://en.wikipedia.org/wiki/University_Library_of_T%C3%BCbingen', 506, 'University'],

INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:28] "GET /static/pictures/748.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:28] "GET /static/pictures/699.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:28] "GET /static/pictures/506.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:29] "GET /static/pictures/703.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:29] "GET /static/pictures/616.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:29] "GET /static/pictures/739.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:29] "GET /static/pictures/746.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:29] "GET /static/pictures/740.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:29] "GET /static/pictures/118.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [21/Jul/2024 21:59:29] "GET /static/pictures/698.svg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [2

# 4. Performance Evaluation 
We will evaluate the performance of our search systems on the basis of five queries. Two of them are avilable to you now for engineering purposes:
- `tübingen attractions`
- `food and drinks`

The remaining three queries will be given to you during our final session on July 23rd. Please be prepared to run your systems and produce a single result file for all five queries live in class. That means you should aim for processing times of no more than ~1 minute per query. We will ask you to send carsten.eickhoff@uni-tuebingen.de that file.

# Grading
Your final projects will be graded along the following criteria:
- 25% Code correctness and quality (to be delivered on this sheet)
- 25% Report (4 pages, PDF, explanation and justification of your design choices)
- 25% System performance (based on how well your system performs on the 5 queries relative to the other teams in terms of nDCG)
- 15% Creativity and innovativeness of your approach (in particular with respect to your search system #2 and user interface #3 innovations)
- 10% Presentation quality and clarity

# Permissible libraries
You can use any general-puprose ML and NLP libraries such as scipy, numpy, scikit-learn, spacy, nltk, but please stay away from dedicated web crawling or search engine toolkits such as scrapy, whoosh, lucene, terrier, galago and the likes. Pretrained models are fine to use as part of your system, as long as they have not been built/trained for retrieval. 
